<a href="https://colab.research.google.com/github/hectorcamachoz/Seleccion_Caracteristicas/blob/main/A1_4_594557.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea A1.4 Seleccion de Caracteristicas

En esta tarea se realizara un modelo de regresion multivariable, con variables previamente filtradas con metodos de seleccion de caracteristicas.

**1.** En el primer bloque de codigo, se imporatara la base de datos, y se imprimira el tamaño y las primeras 5 filas de datos.

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
df = pd.read_csv('Vino_Tinto.csv')
print(df.shape)
print(df.head(5))

(1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        5  
3        6  
4        5

**2.** En el siguiente bloque, se dividira la base de datos, en dos, de manera aleatoria. El 80% sera utilizado para entrenar el modelo, y el 20% para probarlo. Esto se hara con la funcion train_test_split de la libreria sklearn.model_selection

In [68]:
# Para utilizar train_test_split() se importa de sklearn.model_selection
train, test = train_test_split(df, train_size=0.8)
print(train.shape)
print(test.shape)
print(train.shape[0] , '+', test.shape[0] ,'=', int(train.shape[0]) + int(test.shape[0]))


(1279, 12)
(320, 12)
1279 + 320 = 1599


**3.** A continuacion, se realizara una seleccion hacia adelante usando todas las variables, excepto la llamada calidad, ya que esa sera nuestra variable de salida 'y'. Se utilizara la funcion 'SequentialFeatureSelector' de la libreria 'mlxtend.feature_selection'

In [69]:
x_train = train.drop('calidad', axis = 1)
y_train = train.calidad
md = LinearRegression()
sfs = SFS(md,k_features=(2,8),forward=True, scoring='r2',cv=10)
sfs.fit(x_train,y_train)

features = list(sfs.k_feature_names_)
print('Variables seleccionadas: ', features)

Variables seleccionadas:  ['acidezVolatil', 'acidoCitrico', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


**4.** Ahora que se tienen las variables con mayor relacion, de acuerdo al metodo de seleccion de caracteristicas hacia adelante. Se generara un modelo utilizando exclusivamente esas variables. Se utilizara la funcion 'r2_score' de la libreria 'sklearn.metrics', para medir la capacidad de prediccion del modelo.

In [70]:
from sklearn.metrics import r2_score


x_train_1 = train[features] #guardar las variables seleccionadas en x_train_1
md.fit(x_train_1, y_train) #Entrenar el modelo con las nuevas variables

x_test_1 = test[features] #guardar las variables seleccionadas en x_test_1
y_test = test.calidad

y_pred = md.predict(x_test_1) #predecir utilizando el nuevo modelo, utilizando x_test_1

r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

R2:  0.3336042973837816


**5.** A continuacion se hara un metodo de seleccion hacia atras, utilizando las variables previamente seleccionadas por el metodo hacia adelante.

In [71]:

md_2 = LinearRegression()
sfs_2 = SFS(md_2,k_features=(2,5),forward=False, scoring='r2',cv=10)

sfs_2.fit(x_train_1, y_train)
features_2 = list(sfs_2.k_feature_names_)
print('Variables seleccionadas: ', features_2)

Variables seleccionadas:  ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


**6.** Por ultimo, se generara un nuevo modelo con las nuevas variables seleccionadas por el metodo hacia adelante y hacia atras.

In [72]:

x_train_2 = train[features_2] #guardar las variables seleccionadas en x_train_2
md_2.fit(x_train_2, y_train) #Entrenar el modelo con las nuevas variables

x_test_2 = test[features_2] #guardar las variables seleccionadas en x_test_2
y_test = test.calidad

y_pred_2 = md_2.predict(x_test_2) #predecir utilizando el nuevo modelo, utilizando x_test_2

r2 = r2_score(y_test, y_pred_2)
print('R2: ', r2)

R2:  0.32489028726349034


Al observar las R2 de ambos modelos, podemos observar que la primera es mayor a la segunda, y esto no significa que sea mejor. Al momento de usar mas variables la R2 aumenta, entonces no podriamos concluir cual es el mejor modelo, para eso necesitariamos utilizar R2 ajustada, ademas es importante recalcar que en el proceso hemos caido en una fuga de datos, por lo tanto no se puede concluir con certeza cual es el mejor modelo.